In [1]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import gc
import csv

### Книги

In [2]:
source = 'http://studyenglishwords.com'

In [ ]:
page = BeautifulSoup(requests.get(source+'/result/books').text, "html5lib")

In [ ]:
books = page.find("table").find_all("a")

In [ ]:
# русско- и франкоязычные писатели
irrelevant = [0,1,9,32,44,45,53,54,56,66,70,78,84,85,86,91,92,94,96,98,99]

In [ ]:
book_link = {}
for i in range(len(books)):
    if i not in irrelevant:
        book_link[books[i].text] = books[i].get('href')

In [3]:
def replace(string, substitutions):

    substrings = sorted(substitutions, key=len, reverse=True)
    regex = re.compile('|'.join(map(re.escape, substrings)))
    return regex.sub(lambda match: substitutions[match.group(0)], string)

In [4]:
# извлекаем со страницы данные и сохраняем в df через список
def save_page(page, df):
    substitutions = {'\n':' ', '\t':' '}
    en = [replace(sent.find("span").text, substitutions) for sent in page.find_all("div", {"class":"left"})]
    ru = [replace(sent.find("span").text, substitutions) for sent in bookpage.find_all("div", {"class":"right"})]
    start = len(df)
    for i in range(len(en)):
        df.loc[start+i] = [en[i], ru[i]]

In [ ]:
columns = ['en', 'ru']

for book in book_link:
    # открыть книгу
    bookpage = BeautifulSoup(requests.get(source+book_link[book]).text, "html5lib")
    df = pd.DataFrame(columns=columns)
    # пока есть страницы
    while bookpage:
        # сохраняем их в df
        save_page(bookpage, df)
        # получаем следующую страницу
        next_page = bookpage.find("span", {"class":"next"}).find("a")
        if next_page:
            bookpage = BeautifulSoup(requests.get(source+book_link[book]+next_page.get("href")).text, "html5lib")
        else:
            bookpage = False
    # сохранить датафрейм
    df.to_csv(book+'.tsv', sep = '\t', quoting=csv.QUOTE_NONE)
    print(book+' is saved')    
    # чтобы точно очистить память от датафрейма
    del df
    gc.collect()

### Субтитры

In [5]:
# на двух страницах, проще вручную
page_sub1 = BeautifulSoup(requests.get('https://studyenglishwords.com/browse?page=1&type=movie').text, "html5lib")
page_sub2 = BeautifulSoup(requests.get('https://studyenglishwords.com/browse?page=2&type=movie').text, "html5lib")

In [12]:
subs = page_sub1.find_all("a", {"class":"type-common type-movie"})+page_sub2.find_all("a", {"class":"type-common type-movie"})

In [14]:
sub_link = {}
for sub in subs:
    sub_link[re.sub('/', '-', sub.text.strip())] = sub.get('href')

In [15]:
# columns = ['en', 'ru']

for sub in sub_link:
    # открыть книгу
    bookpage = BeautifulSoup(requests.get(source+sub_link[sub]).text, "html5lib")
    df = pd.DataFrame(columns=columns)
    # пока есть страницы
    while bookpage:
        # сохраняем их в df
        save_page(bookpage, df)
        # получаем следующую страницу
        next_page = bookpage.find("span", {"class":"next"}).find("a")
        if next_page:
            bookpage = BeautifulSoup(requests.get(source+sub_link[sub]+next_page.get("href")).text, "html5lib")
        else:
            bookpage = False
    # сохранить датафрейм
    df.to_csv(sub+'.tsv', sep = '\t', quoting=csv.QUOTE_NONE)
    print(sub+' is saved')
    
    del df
    gc.collect()

Дюна - Dune (1984-12-14) is saved
New Moon - New Moon (2010-01-14) is saved
12 Разгневанных мужчин - 12 Angry Men (1957-04-10) is saved
Гарри Поттер и философский камень - Harry Potter and the Philosopher's Stone (2001-11-15) is saved
Форсаж 4 - Fast & Furious (2009-04-03) is saved
Король Лев - The Lion King (1994-06-23) is saved
Интервью с вампиром - Interview with the Vampire (1994-11-11) is saved
Бойцовский клуб - Fight Club (1999-10-14) is saved
1+1 - Неприкасаемые - Intouchables (2011-11-02) is saved
Гордость и предубеждение - Pride & Prejudice (2005-11-11) is saved
Зеленая миля - The Green Mile (1999-12-05) is saved
Терминатор 2: Судный день - Terminator 2: Judgment Day (1991-07-01) is saved
Призрак оперы - The Phantom of the Opera (2004-12-22) is saved
Перевозчик - The Transporter (2002-10-02) is saved
Форрест Гамп - Forrest Gump (1994-06-22) is saved
Гордость и предубеждение - Pride and Prejudice (1995-09-24) is saved
Матрица - The Matrix (1999-03-30) is saved
Сумерки - Twiligh